In [ ]:
from fastdebug.core import *

In [ ]:
import inspect
from inspect import *
from inspect import _signature_from_callable
from inspect import _signature_is_functionlike, _signature_is_builtin, _signature_get_user_defined_method, _signature_from_function, _signature_bound_method

In [ ]:
class Foo:
    def __init__(self, a, b, c): pass

In [ ]:
inspect.signature(Foo)

<Signature (a, b, c)>

In [ ]:
inspect.signature??

Signature: inspect.signature(obj, *, follow_wrapped=True)
Source:   
def signature(obj, *, follow_wrapped=True):
    """Get a signature object for the passed callable."""
    return Signature.from_callable(obj, follow_wrapped=follow_wrapped)
File:      ~/mambaforge/lib/python3.9/inspect.py
Type:      function


In [ ]:
Signature.from_callable??

Signature: Signature.from_callable(obj, *, follow_wrapped=True)
Source:   
    @classmethod
    def from_callable(cls, obj, *, follow_wrapped=True):
        """Constructs Signature for the given callable object."""
        return _signature_from_callable(obj, sigcls=cls,
                                        follow_wrapper_chains=follow_wrapped)
File:      ~/mambaforge/lib/python3.9/inspect.py
Type:      method


In [ ]:
_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

<Signature (a, b, c)>

### give dbprint the current main globals() is very crucial

In [ ]:
g = globals() # this is a crucial step
# 1. it brings everything _signature_from_callable needed from inspect.py 
# 2. it allows the new srcode function override the original official srcode in outer scope, 
# so that you can call the new function with the same name _signature_from_callable

### Try one srcline with one investigation

In [ ]:
sfc = dbprint(_signature_from_callable, "if isinstance(obj, type):", "this is comment", \
              "locals()", "isinstance(obj, type)",\
              expand=1, env=g)

_signature_from_callable(Foo, sigcls=inspect.Signature, follow_wrapper_chains=True) # debug the function above through this example

    sig = None                                                                                                                                          (90)
    if isinstance(obj, type):===========================================================================================================================(91)
                                                                                                                                              this is comment
        # obj is a class or a metaclass                                                                                                                 (92)


locals() => locals() : {'env': {'obj': <class '__main__.Foo'>, 'follow_wrapper_chains': True, 'skip_bound_arg': True, 'sigcls': <class 'inspect.Signature'>, '_get_signature_of': functools.partial(<function _signature_from_callable>, follow_wrapper_chains=True, skip_bound_arg=True, sigcls=<class 'inspect.Signature'>), 'sig': None}, 'codes': ('locals()', 'isinstance(ob

<Signature (a, b, c)>

In [ ]:
inspect.signature(Foo)

<Signature (a, b, c)>

In [ ]:
# this way we are accessing the original source from inspect.py
inspect._signature_from_callable?

Signature:
inspect._signature_from_callable(
    obj,
    *,
    follow_wrapper_chains=True,
    skip_bound_arg=True,
    sigcls,
)
Docstring:
Private helper function to get signature for arbitrary
callable objects.
File:      ~/mambaforge/lib/python3.9/inspect.py
Type:      function


In [ ]:
# this is our dbsrc in main.py here
_signature_from_callable?

Signature:
_signature_from_callable(
    obj,
    *,
    follow_wrapper_chains=True,
    skip_bound_arg=True,
    sigcls,
)
Docstring:
Private helper function to get signature for arbitrary
callable objects.
File:      Dynamically generated function. No source code available.
Type:      function


### make `inspect.signature` to run our dbsrc code
overriding the original official in inspect library

In [ ]:
sfc = dbprint(_signature_from_callable, "if isinstance(obj, type):", "this is comment", \
              "locals()", "isinstance(obj, type)", \
              expand=1, env=g)

# overriding the original official source with our dbsrc, even though rewriting _signature_from_callable inside inspect.py ######################
inspect._signature_from_callable = _signature_from_callable
inspect.signature(Foo) 

    sig = None                                                                                                                                          (90)
    if isinstance(obj, type):===========================================================================================================================(91)
                                                                                                                                              this is comment
        # obj is a class or a metaclass                                                                                                                 (92)


locals() => locals() : {'env': {'obj': <class '__main__.Foo'>, 'follow_wrapper_chains': True, 'skip_bound_arg': True, 'sigcls': <class 'inspect.Signature'>, '_get_signature_of': functools.partial(<function _signature_from_callable>, follow_wrapper_chains=True, skip_bound_arg=True, sigcls=<class 'inspect.Signature'>), 'sig': None}, 'codes': ('locals()', 'isinstance(ob

<Signature (a, b, c)>

### run dbcode which are pure simple expressions

In [ ]:
sfc = dbprint(_signature_from_callable, "call = _signature_get_user_defined_method(type(obj), '__call__')", "this is comment", \
              "locals()", "_signature_get_user_defined_method(type(obj), '__call__')", "inspect.getdoc(_signature_get_user_defined_method)", expand=5, env=g)

inspect._signature_from_callable = _signature_from_callable
inspect.signature(Foo) 

    if isinstance(obj, type):                                                                                                                           (91)
        # obj is a class or a metaclass                                                                                                                 (92)
                                                                                                                                                        (93)
        # First, let's see if it has an overloaded __call__ defined                                                                                     (94)
        # in its metaclass                                                                                                                              (95)
        call = _signature_get_user_defined_method(type(obj), '__call__')================================================================================(96)
                                                          

<Signature (a, b, c)>

### run dbcode involving assignament = e.g., pprint(..., width=157)

In [ ]:
sfc = dbprint(_signature_from_callable, 100, "this is comment", \
              "pprint(locals(), width=157)", expand=1, env=g)

inspect._signature_from_callable = _signature_from_callable
inspect.signature(Foo) 

        else:                                                                                                                                           (99)
            factory_method = None=======================================================================================================================(100)
                                                                                                                                              this is comment
            new = _signature_get_user_defined_method(obj, '__new__')                                                                                    (101)


{'c': 'pprint(locals(), width=157)',
 'codes': ('pprint(locals(), width=157)',),
 'env': {'_get_signature_of': functools.partial(<function _signature_from_callable>, follow_wrapper_chains=True, skip_bound_arg=True, sigcls=<class 'inspect.Signature'>),
         'call': None,
         'follow_wrapper_chains': True,
         'obj': <class '__main__.Foo'>,
         'si

<Signature (a, b, c)>

In [ ]:
list(locals().items())[:5]


### run dbcode with for in loop and use idx instead of srcode string

In [ ]:
sfc = dbprint(_signature_from_callable, 102, "this is comment", \
"loc = list(locals().items())[:5]\\n\
for k, v in loc:\\n\
    print(f'{k}: {v}')", expand=1, env=g)

inspect._signature_from_callable = _signature_from_callable
inspect.signature(Foo) 

            new = _signature_get_user_defined_method(obj, '__new__')                                                                                    (101)
            init = _signature_get_user_defined_method(obj, '__init__')==================================================================================(102)
                                                                                                                                              this is comment
            # Now we check if the 'obj' class has an own '__new__' method                                                                               (103)


loc = list(locals().items())[:5]
for k, v in loc:
    print(f'{k}: {v}')                                                                                     
                                                                                                                                  Running your code block => 
The code block printout => : 
env: {'obj': <class 

<Signature (a, b, c)>

### test printsrcwithidx

In [ ]:
printsrcwithidx(defaults.orisrc)

def _signature_from_callable(obj, *,                                                                                                                    (0)
                             follow_wrapper_chains=True,                                                                                                (1)
                             skip_bound_arg=True,                                                                                                       (2)
                             sigcls):                                                                                                                   (3)
                                                                                                                                                        (4)
    """Private helper function to get signature for arbitrary                                                                                           (5)
    callable objects.                                           